本次以英雄联盟对局胜负预测任务为基础，实现决策树算法相关细节，加深对算法的理解，并了解做机器学习任务的大致流程。

### 实验介绍
英雄联盟（League of Legends，LoL）是一个多人在线竞技游戏，由拳头游戏（Riot Games）公司出品。在游戏中，每位玩家控制一位有独特技能的英雄，红蓝两支队伍各有五位玩家进行对战，目标是摧毁对方的基地水晶。水晶有多座防御塔保护，通常需要先摧毁一些防御塔再摧毁水晶。玩家所控制的英雄起初非常弱，需要不断击杀小兵、野怪和对方英雄来获得金币、经验。经验可以提升英雄等级和技能等级，金币可以用来购买装备提升攻击、防御等属性。对战过程中一般没有己方单位在附近的地点是没有视野的，即无法看到对面单位，双方可以通过使用守卫来监视某个地点，洞察对面走向、制定战术。
本数据集来自[Kaggle](https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min)，包含了9879场钻一到大师段位的单双排对局，对局双方几乎是同一水平。每条数据是前10分钟的对局情况，每支队伍有19个特征，红蓝双方共38个特征。这些特征包括英雄击杀、死亡，金钱、经验、等级情况等等。一局游戏一般会持续30至40分钟，但是实际前10分钟的局面很大程度上影响了之后胜负的走向。作为最成功的电子竞技游戏之一，对局数据、选手数据的量化与研究具有重要意义，可以启发游戏将来的发展和改进。
根据已有的对局前10分钟特征信息，预测最后获胜方是蓝色方还是红色方。

In [1]:
from collections import Counter
import pandas as pd # 数据处理
import numpy as np # 数学运算
from sklearn.model_selection import train_test_split, cross_validate # 划分数据集函数
from sklearn.metrics import accuracy_score # 准确率函数
RANDOM_SEED = 2020 # 固定随机种子

### 读入数据

In [2]:
csv_data = './data/high_diamond_ranked_10min.csv' # 数据路径
data_df = pd.read_csv(csv_data, sep=',') # 读入csv文件为pandas的DataFrame
data_df = data_df.drop(columns='gameId') # 舍去对局标号列

###  数据概览

In [3]:
print(data_df.iloc[0]) # 输出第一行数据
data_df.describe() # 每列特征的简单统计信息

blueWins                            0.0
blueWardsPlaced                    28.0
blueWardsDestroyed                  2.0
blueFirstBlood                      1.0
blueKills                           9.0
blueDeaths                          6.0
blueAssists                        11.0
blueEliteMonsters                   0.0
blueDragons                         0.0
blueHeralds                         0.0
blueTowersDestroyed                 0.0
blueTotalGold                   17210.0
blueAvgLevel                        6.6
blueTotalExperience             17039.0
blueTotalMinionsKilled            195.0
blueTotalJungleMinionsKilled       36.0
blueGoldDiff                      643.0
blueExperienceDiff                 -8.0
blueCSPerMin                       19.5
blueGoldPerMin                   1721.0
redWardsPlaced                     15.0
redWardsDestroyed                   6.0
redFirstBlood                       0.0
redKills                            6.0
redDeaths                           9.0


blueWins  blueWardsPlaced  blueWardsDestroyed  blueFirstBlood  \
count  9879.000000      9879.000000         9879.000000     9879.000000   
mean      0.499038        22.288288            2.824881        0.504808   
std       0.500024        18.019177            2.174998        0.500002   
min       0.000000         5.000000            0.000000        0.000000   
25%       0.000000        14.000000            1.000000        0.000000   
50%       0.000000        16.000000            3.000000        1.000000   
75%       1.000000        20.000000            4.000000        1.000000   
max       1.000000       250.000000           27.000000        1.000000   

         blueKills   blueDeaths  blueAssists  blueEliteMonsters  blueDragons  \
count  9879.000000  9879.000000  9879.000000        9879.000000  9879.000000   
mean      6.183925     6.137666     6.645106           0.549954     0.361980   
std       3.011028     2.933818     4.064520           0.625527     0.480597   
min       0.000000     0.000000     0.000000           0.000000     0.000000   
25%       4.000000     4.000000     4.000000           0.000000     0.000000   
50%       6.000000     6.000000     6.000000           0.000000     0.000000   
75%       8.000000     8.000000     9.000000           1.000000     1.000000   
max      22.000000    22.000000    29.000000           2.000000     1.000000   

       blueHeralds  ...  redTowersDestroyed  redTotalGold  redAvgLevel  \
count  9879.000000  ...         9879.000000   9879.000000  9879.000000   
mean      0.187974  ...            0.043021  16489.041401     6.925316   
std       0.390712  ...            0.216900   1490.888406     0.305311   
min       0.000000  ...            0.000000  11212.000000     4.800000   
25%       0.000000  ...            0.000000  15427.500000     6.800000   
50%       0.000000  ...            0.000000  16378.000000     7.000000   
75%       0.000000  ...            0.000000  17418.500000     7.200000   
max       1.000000  ...            2.000000  22732.000000     8.200000   

       redTotalExperience  redTotalMinionsKilled  redTotalJungleMinionsKilled  \
count         9879.000000            9879.000000                  9879.000000   
mean         17961.730438             217.349226                    51.313088   
std           1198.583912              21.911668                    10.027885   
min          10465.000000             107.000000                     4.000000   
25%          17209.500000             203.000000                    44.000000   
50%          17974.000000             218.000000                    51.000000   
75%          18764.500000             233.000000                    57.000000   
max          22269.000000             289.000000                    92.000000   

        redGoldDiff  redExperienceDiff  redCSPerMin  redGoldPerMin  
count   9879.000000        9879.000000  9879.000000    9879.000000  
mean     -14.414111          33.620306    21.734923    1648.904140  
std     2453.349179        1920.370438     2.191167     149.088841  
min   -11467.000000       -8348.000000    10.700000    1121.200000  
25%    -1596.000000       -1212.000000    20.300000    1542.750000  
50%      -14.000000          28.000000    21.800000    1637.800000  
75%     1585.500000        1290.500000    23.300000    1741.850000  
max    10830.000000        9333.000000    28.900000    2273.200000  

[8 rows x 39 columns]

### 增删特征
传统的机器学习模型大部分都是基于特征的，因此特征工程是机器学习中非常重要的一步。有时构造一个好的特征比改进一个模型带来的提升更大。这里简单展示一些特征处理的例子。首先，上面提到，特征列中有些特征信息是完全冗余的，会给模型带来不必要的计算量，可以去除。其次，相比于红蓝双方击杀、助攻的绝对值，可能双方击杀英雄的差值更能体现出当前对战的局势。因此，我们可以构造红蓝双方对应特征的差值。数据文件中已有的差值是金币差GoldDiff和经验差ExperienceDiff，实际上每个对应特征都可以构造这样的差值特征。

In [4]:
drop_features = ['blueGoldDiff', 'redGoldDiff', 
                 'blueExperienceDiff', 'redExperienceDiff', 
                 'blueCSPerMin', 'redCSPerMin', 
                 'blueGoldPerMin', 'redGoldPerMin'] # 需要舍去的特征列
df = data_df.drop(columns=drop_features) # 舍去特征列
info_names = [c[3:] for c in df.columns if c.startswith('red')] # 取出要作差值的特征名字（除去red前缀）
print(type(info_names))
print(info_names)
for info in info_names: # 对于每个特征名字
    df['br' + info] = df['blue' + info] - df['red' + info] # 构造一个新的特征，由蓝色特征减去红色特征，前缀为br
# 其中FirstBlood为首次击杀最多有一只队伍能获得，brFirstBlood=1为蓝，0为没有产生，-1为红
df = df.drop(columns=['blueFirstBlood', 'redFirstBlood']) # 原有的FirstBlood可删除

print(df.info())
print(df.describe())


<class 'list'>
['WardsPlaced', 'WardsDestroyed', 'FirstBlood', 'Kills', 'Deaths', 'Assists', 'EliteMonsters', 'Dragons', 'Heralds', 'TowersDestroyed', 'TotalGold', 'AvgLevel', 'TotalExperience', 'TotalMinionsKilled', 'TotalJungleMinionsKilled']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9879 entries, 0 to 9878
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   blueWins                      9879 non-null   int64  
 1   blueWardsPlaced               9879 non-null   int64  
 2   blueWardsDestroyed            9879 non-null   int64  
 3   blueKills                     9879 non-null   int64  
 4   blueDeaths                    9879 non-null   int64  
 5   blueAssists                   9879 non-null   int64  
 6   blueEliteMonsters             9879 non-null   int64  
 7   blueDragons                   9879 non-null   int64  
 8   blueHeralds                   9879 non-null   int64  

### 特征离散化
决策树ID3算法一般是基于离散特征的，本例中存在很多连续的数值特征，例如队伍金币。直接应用该算法每个值当作一个该特征的一个取值可能造成严重的过拟合，因此需要对特征进行离散化，即将一定范围内的值映射成一个值，例如对用户年龄特征，将0-10映射到0，11-18映射到1，19-25映射到2，25-30映射到3，等等类似，然后在决策树构建时使用映射后的值计算信息增益。

In [5]:
discrete_df = df.copy() # 先复制一份数据

for c in df.columns[1:]: # 遍历每一列特征，跳过标签列
    kind_num = df[c].nunique()
    if kind_num > 30:
        discrete_df[c] = pd.qcut(df[c],q=43,labels = False, duplicates='drop')
df.nunique()

blueWins                           2
blueWardsPlaced                  147
blueWardsDestroyed                27
blueKills                         21
blueDeaths                        21
blueAssists                       30
blueEliteMonsters                  3
blueDragons                        2
blueHeralds                        2
blueTowersDestroyed                5
blueTotalGold                   4739
blueAvgLevel                      17
blueTotalExperience             4143
blueTotalMinionsKilled           148
blueTotalJungleMinionsKilled      74
redWardsPlaced                   151
redWardsDestroyed                 25
redKills                          21
redDeaths                         21
redAssists                        28
redEliteMonsters                   3
redDragons                         2
redHeralds                         2
redTowersDestroyed                 3
redTotalGold                    4732
redAvgLevel                       18
redTotalExperience              4113
r

### 数据集准备

In [6]:
all_y = discrete_df['blueWins'].values # 所有标签数据
feature_names = discrete_df.columns[1:] # 所有特征的名称
all_x = discrete_df[feature_names].values # 所有原始特征值，pandas的DataFrame.values取出为numpy的array矩阵

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(all_x, all_y, test_size=0.2, random_state=RANDOM_SEED)
all_y.shape, all_x.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape # 输出数据行列信息


"\nm = np.arange(30).reshape(10,3)\nn = [1,1,0,1,0,0,1,1,0,1]\nc = ['a','b','c']\na = pd.DataFrame(m, columns=c)\na.insert(0,'target', n)\nprint(a)\nall_y = a['target'].values\nfeature_names = a.columns[1:]\nall_x = a[feature_names].values\nx_train, x_test, y_train, y_test = train_test_split(all_x, all_y, test_size=1, random_state=RANDOM_SEED)\nall_y.shape, all_x.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape # 输出数据行列信息\n"

###  决策树模型的实现

In [7]:
import math
# 定义决策树类
class DecisionTree(object):
    def __init__(self, classes, features, 
                 max_depth=10, min_samples_split=10,
                 impurity_t='entropy'):

        self.classes = classes
        self.features = features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.impurity_t = impurity_t
        self.root = {} # 定义根节点，未训练时为空, 以字典的方式存储整个树
        # self define
        self.i = 0
        self.it = 0
        

    def impurity(self, kind_num): # kind_num is series
        #print(kind_num, end = ':')
        #print(kind_num.nunique())
        if self.impurity_t == 'gini':
            pp_sum = 0 # probablity pow sum
            for i,j in kind_num.iteritems(): # i is index, j is value
                pp_sum += math.pow((j / sum(kind_num)), 2)
            return 1-pp_sum
        elif self.impurity_t == 'entropy':
            plogp_sum = 0
            for i,j in kind_num.iteritems():
                p = j / sum(kind_num)
                plogp_sum += p * math.log(p, 2)
            return -plogp_sum
        elif self.impurity_t == 'wrongclass':
            maxj = 0
            for i,j in kind_num.iteritems():
                p = j / sum(kind_num)
                if maxj < p:
                    maxj = p
            return 1-maxj
        else:
            print("Not support!!!")
            
        
    def gain(self,S,A): # S is df, A is S's kind
        g = 0  # for sum of child gini
        Tvc = S[ 'target' ].value_counts()
        #print(Tvc)
        Tvc_num = sum( Tvc )  # number of tag in this level
        IA = self.impurity(S[ 'target' ].value_counts())
        for k in S[A].unique():
            tvc = S[ S[A] == k ][ 'target' ].value_counts()
            tvc_num = sum(tvc)
            IAK = self.impurity(tvc)
            g += IAK * tvc_num / Tvc_num
        return IA - g
        
    def tree(self, df, Feature, k, d):
        if len(df) <= self.min_samples_split or df['target'].nunique() == 1 or d > self.max_depth:
            vc = df['target'].value_counts()
            mv = 0 #max value
            for i,j in vc.iteritems():
                if mv < j:
                    mv = j   # max value of target
                    mk = i   # the kind of max value
            return mk
        ig = -1 # gain list in same level
        # this loop just computer this level gain max
        self.i = self.i + 1
        for f in df.iloc[:,1:].columns:
            if ig < self.gain(df, f) and len(df) > self.min_samples_split:
                ig = self.gain(df, f)
                F = f 
        d = d + 1
        tmp_dict = {}
        i = 0
        for k in df[F].unique():
            i = i+1
            tree_return = {}
            tree_return = self.tree(df[df[F]==k], F, k, d)
            tmp_dict.update({k:tree_return})
        tmp_dict = {F:tmp_dict}
        return tmp_dict
            
        
            
    def fit(self, feature, label ):
        assert len(self.features) == len(feature[0]) # 输入数据的特征数目应该和模型定义时的特征数目相同
        df = pd.DataFrame(feature, columns=self.features)
        df.insert(0, 'target', label)
        self.root = self.tree(df,'first','first', 0)
        print('fit finished')

        
    def route_tree(self, row, root):
        self.it = self.it + 1
        for feature in row.index:
            if feature not in root:
                continue
            if root[feature] == 0 or root[feature] == 1 :
                return root[feature]
            if row[feature] not in root[feature]:
                i = 1
                r = row[feature]
                while row[feature]-i not in root[feature] and row[feature]+i not in root[feature]:
                    i = i + 1
                row[feature] = row[feature]-i if row[feature]-i in root[feature] else row[feature]+i
            if root[feature][row[feature]] == 0 or root[feature][row[feature]] == 1 :
                return root[feature][row[feature]]
            else:
                return self.route_tree(row, root[feature][row[feature]])
        
    
    def predict(self, feature):
        assert len(feature.shape) == 1 or len(feature.shape) == 2 # 只能是1维或2维
        df = pd.DataFrame(feature, columns=self.features)
        return [self.route_tree(row, self.root) for i,row in df.iterrows()]

In [8]:
# 定义决策树模型，传入算法参数
#DT = DecisionTree(classes=[0,1], features=feature_names, max_depth=5, min_samples_split=10, impurity_t='gini')
DT = DecisionTree(classes=[0,1], features=feature_names, max_depth=5, min_samples_split=198, impurity_t='entropy')
#DT = DecisionTree(classes=[0,1], features=feature_names, max_depth=2, min_samples_split=30, impurity_t='entropy')
#DT = DecisionTree(classes=[0,1], features=feature_names, max_depth=5, min_samples_split=10, impurity_t='wrongclass')

DT.fit(x_train, y_train) # 在训练集上训练
p_test = DT.predict(x_test) # 在测试集上预测，获得预测值
print(DT.root)
#print(p_test) # 输出预测值
#test_acc = accuracy_score(p_test, y_test) # 将测试预测值与测试集标签对比获得准确率
#print('accuracy: {:.4f}'.format(test_acc)) # 输出准确率
p_train = DT.predict(x_train)
train_acc = accuracy_score(p_train, y_train)
print('train accuracy: {:.4f}'.format(train_acc))

fit finished
{'brTotalGold': {39.0: 1, 4.0: 0, 14.0: 0, 15.0: 0, 21.0: 0, 12.0: 0, 20.0: 0, 32.0: 1, 33.0: 1, 40.0: 1, 11.0: 0, 6.0: 0, 7.0: 0, 22.0: 1, 23.0: 1, 38.0: 1, 24.0: 1, 41.0: 1, 35.0: 1, 2.0: 0, 1.0: 0, 42.0: 1, 37.0: 1, 10.0: 0, 29.0: 1, 13.0: 0, 30.0: 1, 31.0: 1, 9.0: 0, 16.0: 0, 17.0: 0, 26.0: 1, 3.0: 0, 19.0: 0, 36.0: 1, 27.0: 1, 28.0: 1, 34.0: 1, 5.0: 0, 8.0: 0, 0.0: 0, 18.0: 0, 25.0: 1}}
train accuracy: 0.7271


### 模型调优
第一次模型测试结果可能不够好，可以先检查调试代码是否有bug，再尝试调整参数或者优化计算方法。

### 总结
一个完整的机器学习任务包括：确定任务、数据分析、特征工程、数据集划分、模型设计、模型训练和效果测试、结果分析和调优等多个阶段，本案例以英雄联盟游戏胜负预测任务为例，给出了每个阶段的一些简单例子，帮助大家入门机器学习，希望大家有所收获！

In [9]:
print(p_test)
print(len(p_test))
test_acc = accuracy_score(p_test, y_test) # 将测试预测值与测试集标签对比获得准确率
print('accuracy: {:.4f}'.format(test_acc)) # 输出准确率


[0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 